# 作业二：关联规则挖掘
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;**秦馨**
<br>&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;**5720182058**

# 2.1 数据预处理

### 数据集说明
 数据源：mlb_pitch/atbats.csv，来自2015-2018年MLB常规赛期间每投一球的球场级别数据，包含10个属性和740k行的数据
* ab_id：at-bat ID.前四位是年份
* batter_id：击球手的球员编号，由MLB提供
* event：击球结果的描述
* g_id：游戏ID，前四位数字是年份
* inning：局数
* o：在这个打击后出局的次数
* p_score：为投手队得分
* p_throws：哪个投手投球，单个字符，R或L
* pitcher_id：投手的球员编号，由MLB提供
* stand：哪边击球手击球了，单个字符，R或L
* top：如果是在上半局为真，如果是在下半局为假

### 取样
由于数据集过于庞大，限制于电脑配置与处理能力，从中抽取部分样本数据进行关联规则挖掘
* 设定样本数为：10000

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 对数据集取样
def samples_mlb(work_data):
    data_index= work_data[['ab_id', 'batter_id', 'event', 'inning', 'o', 'p_score', 'pitcher_id', 'p_throws', 'stand', 'top']]
    data_new=data_index.head(10000)
    data_new.to_csv('C:/Users/xin/Desktop/数据挖掘与处理/mlb-pitch-data-20152018/atbats_samples.csv')

In [ ]:
file_1=pd.read_csv('C:/Users/xin/Desktop/数据挖掘与处理/mlb-pitch-data-20152018/atbats.csv', engine='python')     # 打开文件
samples_mlb(file_1)

### 数据预处理
由于数据集中'batter_id', 'inning', 'o', 'p_score', 'pitcher_id', 'p_throws', 'stand'七个属性的所有可能取值之间存在重叠部分甚至完全相同，为了便于进行关联规则的挖掘，需要将属性数据进行处理。将其属性值与属性标签进行组合，即将其转换为适合关联规则挖掘的形式。<br>
如：p_throws 属性中取值为L的属性值转换为 p_throws_L

In [ ]:
# 组合属性名称及属性值
def add_table(index,name):
    for i in range(0,len(index)):
        index[i]=name+'_'+str(index[i])      # 将标签与属性值组合
       # print(index[i])
    return index

In [ ]:
# 数据预处理
def mlb_pitch_process(work_data):
    data_mlb = work_data[['ab_id', 'batter_id', 'event', 'inning', 'o', 'p_score', 'pitcher_id', 'p_throws', 'stand', 'top']]
    data_new = pd.DataFrame(columns=['ab_id', 'batter_id', 'event', 'inning', 'o', 'p_score', 'pitcher_id', 'p_throws', 'stand', 'top'])
    data_batter_id = data_mlb['batter_id']
    data_new['batter_id'] = add_table(data_batter_id, 'batter_id')
    data_inning = data_mlb['inning']
    data_new['inning'] = add_table(data_inning, 'inning')
    data_o = data_mlb['o']
    data_new['o'] = add_table(data_o, 'o')
    data_p_score = data_mlb['p_score']
    data_new['p_score'] = add_table(data_p_score, 'p_score')
    data_pitcher_id = data_mlb['pitcher_id']
    data_new['pitcher_id'] = add_table(data_pitcher_id, 'pitcher_id')
    data_p_throws = data_mlb['p_throws']
    data_new['p_throws'] = add_table(data_p_throws, 'p_throws')
    data_stand = data_mlb['stand']
    data_new['stand'] = add_table(data_stand, 'stand')
    data_new['event'] = data_mlb['event']                              # 本身不影响关联规则挖掘的属性使用原始数据
    data_new['ab_id'] = data_mlb['ab_id']
    data_new['top'] = data_mlb['top']

    data_new.to_csv('C:/Users/xin/Desktop/数据挖掘与处理/mlb-pitch-data-20152018/atbats_process.csv')

In [ ]:
file_2 = pd.read_csv('C:/Users/xin/Desktop/数据挖掘与处理/mlb-pitch-data-20152018/atbats_samples.csv', engine='python')
mlb_pitch_process(file_2)

# 2.2 挖掘关联规则

## 2.2.1 Apriori算法
该算法的基本思想是：首先找出所有的频集，这些项集出现的频繁性至少和预定义的最小支持度一样。然后由频集产生强关联规则，这些规则必须满足最小支持度和最小可信度。然后使用第1步找到的频集产生期望的规则，产生只包含集合的项的所有规则，其中每一条规则的右部只有一项，这里采用的是中规则的定义。一旦这些规则被生成，那么只有那些大于用户给定的最小可信度的规则才被留下来。为了生成所有频集，使用了递归的方法。

In [ ]:
# 连接函数
def connect_string(x, ms):                                                                
    x = list(map(lambda i:sorted(i.split(ms)), x))                                       # lambda定义匿名函数
    l = len(x[0])
    r = []
    for i in range(len(x)):
        for j in range(i,len(x)):
            if x[i][:l-1] == x[j][:l-1] and x[i][l-1] != x[j][l-1]:
                r.append(x[i][:l-1]+ sorted([x[j][l-1],x[i][l-1]]))
    return r

In [ ]:
# 寻找关联规则
def find_rule(d, support, confidence, ms=u'--'):                                         # 自定义寻找关联规则的函数
    result = pd.DataFrame(index=['support', 'confidence'])                             # 定义输出结果
    temp = pd.DataFrame(index=['support', 'confidence'])
    score = pd.DataFrame(columns=['lift','kulc','IR'])
    cofidence_series = pd.Series()                                                      # 定义置信度序列
    support_series = 1.0 * d.sum() / len(d)                                                # 支持度序列
    column = list(support_series[support_series > support].index)                          # 初步根据支持度筛选
  #  print(column)
  #  print(support_series)
    k = 0
    column3=[]

    while len(column) > 1:
        k = k + 1
        print(u'\n正在进行第%s次搜索...' % k)                                           # 输出正在搜索次数
        column = connect_string(column, ms)
        print(u'数目：%s...' % len(column))                                              # 输出搜索数目
        sf = lambda i: d[i].prod(axis=1, numeric_only=True)                              # 新一批支持度的计算函数

        d_2 = pd.DataFrame(list(map(sf, column)), index=[ms.join(i) for i in column]).T   # 创建连接数据

        support_series_2 = 1.0 * d_2[[ms.join(i) for i in column]].sum() / len(d)          # 计算连接后的支持度
        column = list(support_series_2[support_series_2 > support].index)                   # 新一轮支持度筛选
        support_series = support_series.append(support_series_2)            
       # print(support_series)
        column2 = []

        for i in column:                                                # 遍历可能的推理，如{A,B,C}究竟是A+B-->C还是B+C-->A还是C+A-->B？
            i = i.split(ms)
            column3.append(i)
           # print(i)
            for j in range(len(i)):
                column2.append(i[:j] + i[j + 1:] + i[j:j + 1])
           # print(column2)
       # print(column3)
       # cofidence_series = pd.Series(index=[ms.join(i) for i in column2])                  
      
    ' # 计算置信度序列'
        for i in column2:                                                                   
            cofidence_series[ms.join(i)] = support_series[ms.join(sorted(i))] / support_series[ms.join(i[:len(i) - 1])]
           # print(i)
           # print(ms.join(i))
        #print(cofidence_series)
        
    '# 计算评价指标'
        n=0
        m=0
        for i in column3:                                                                
            #print(ms.join(i))
            score.loc[ms.join(i)]=0.0
            p = (cofidence_series[n]+cofidence_series[n+1])/2
            score.loc[ms.join(i), 'kulc']= p
            q = abs(support_series[column3[m][0]]-support_series[column3[m][1]])/(support_series[column3[m][0]]
                +support_series[column3[m][1]]-support_series[ms.join(i)])
            score.loc[ms.join(i), 'IR'] = q
            r=cofidence_series[ms.join(i)]/support_series[column3[m][1]]
            score.loc[ms.join(i), 'lift'] = r
            n=n+2
            m=m+1
       # print(score)
    
    '# 根据支持度与置信度寻找关联规则'
        for i in cofidence_series[cofidence_series > confidence].index:                     # 置信度筛选
            result[i] = 0.0
            result[i]['confidence'] = cofidence_series[i]                     
            result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]
    print(u'\n频繁项集为：')
    print(column3)                                                                          # 输出频繁项集
    print(u'\n评价指标为：')
    print(score)                                                                               # 输出评价指标
   # print(cofidence_series)
   # print(support_series)
    result = result.T.sort_values(['confidence', 'support'], ascending=False)           
    print(u'\n关联规则为：')
    print(result)                                                                         # 输出关联规则

    return result,column3

In [ ]:
file = pd.read_csv('C:/Users/xin/Desktop/数据挖掘与处理/mlb-pitch-data-20152018/atbats_process.csv',engine='python',header=None)

    print(u'\n转换原始数据矩阵至0-1矩阵...')                             
    ct = lambda x: pd.Series(1, index=x[pd.notnull(x)])                  # 转换0-1矩阵的过度函数，即将标签数据转换为1
    b = map(ct, file.as_matrix())                                          # 用map方式执行
    file = pd.DataFrame(list(b)).fillna(0)                               # 实现矩阵转换，除了1外其余为空，空值用0填充
    print(u'\n转换完毕')                                            
    del b                                                                     # 删除中间变量b，节省内存

    support = 0.3                                                            # 设置支持度阈值
    confidence = 0.5                                                         # 设置置性度阈值
    ms = '---'                                                               # 连接符设置为---，用来区分不同元素

    print(u'\n开始搜索关联规则...')                                                 
    rule,column3 = find_rule(file, support, confidence, ms)
    rule.to_excel('C:/Users/xin/Desktop/数据挖掘与处理/mlb-pitch-data-20152018/rule.xls')   #  保存关联规则
   

## 2.2.1 寻找频繁项集
* 指定最小支持度 support=0.3

找出交易数据库中所有大于或等于最小支持度的频繁项集：

|mlb_pitch| support |
|:------ | :- - -: |
|'p_throws_R', 'stand_L'|0.348665|
|'True', 'p_throws_R'|0.391261|
|'p_throws_R', 'stand_R'|0.407059|
|'False', 'p_throws_R'|0.364464|

其含义分别为：‘右边投球手投球，左边击球手击球’ ‘上半局，右边投球手投球’ ‘右边投球手投球，右边击球手击球’ ‘下半局，右边投球手投球’

## 2.2.2 导出关联规则
* 支持度：对于项集A，必有A⊆T，记 A.count为交易集D中包含A的交易的数量，则项集A的支持度为：

&emsp;&emsp;&emsp;&emsp;<font size=5>$support(A)=\frac{A.{count}}{|D|}$</font>

关联规则R的支持度是交易集同时包含A和B的交易数与|D|之比。即：

&emsp;&emsp;&emsp;&emsp;<font size=5>$support(A\to B)=\frac{(A\cap B).count}{|D|}$</font>

支持度反映了A、B同时出现的概率,关联规则的支持度等于频繁集的支持度。
* 置信度：对于关联规则R，可信度是指包含A和B的交易数与包含A的交易数之比。即：

&emsp;&emsp;&emsp;&emsp;<font size=5>$confidence(A\to B)=\frac{support(A\to B)}{support(A)}$</font>

可信度反映了如果交易中包含A，则交易包含B的概率。一般来说，只有支持度和可信度较高的关联规则才是用户感兴趣的。
* 指定最小支持度 confidence=0.5

利用频繁项集生成所需要的关联规则，根据设定的最小可信度筛选出强关联规则，如下表所示：

|mlb_pitch| support |confidence|
|- - -|- - -|- - -|
|stand_L---p_throws_R|0.348665|0.821631|
|True---p_throws_R|0.391261|0.769216|
|False---p_throws_R|0.364463|0.741909|
|stand_R---p_throws_R|0.407059|0.707262|
|p_throws_R---stand_R |0.407059|0.538635|
|p_throws_R---True|0.391261|0.517730|

## 2.2.3 评价指标
* Lift（相关性系数）：提升度反映了“项集A的出现”对项集B的出现概率发生了多大的变化,如果该值=1,说明两个条件没有任何关联,如果<1,说明A条件(或者说A事件的发生)与B事件是相斥的。

&emsp;&emsp;&emsp;&emsp;<font size=5>$lift(A,B)=\frac{confidence(A\to B))}{support(B)}=\frac{confidence(B\to A))}{support(A)}$</font>

* Kulc：表示将两种事件作为条件的置信度的均值,避开了支持度的计算,因此不会受零和事务的影响。

&emsp;&emsp;&emsp;&emsp;<font size=5>$kulc(A,B)=\frac{confidence(A\to B)+confidence(B\to A)}{2}$</font>

* IR（不平衡因子）：表示关联规则中两个事物的关联关系的平衡度。

&emsp;&emsp;&emsp;&emsp;<font size=5>$IR(A,B)=\frac{|support(A)-support(B)|}{(support(A)+support(B)-support(A\cap B))}$</font>

对找出规则的评价如下表所示：

|   mlb_pitch  |    lift |   kulc  |    IR|
|- - -|- - -|- - -|- - -|
|p_throws_R---stand_L | 1.087209 | 0.641498 | 0.398557|
|True---p_throws_R  |   1.017852 | 0.643473 | 0.282982|
|p_throws_R---stand_R | 0.935873 | 0.622948 | 0.194958|
|False---p_throws_R  |  0.981719 | 0.612090 | 0.299683|

分析表格可得：
* 提升度lift所得值均约等于一，则说明两个事物几乎没有关联
* kulc系数均为0.6左右，关联度不是很好
* 不平衡因子IR都较低，说明事务间关联关系非常不平衡，不论是左边击球手击球还是右边击球手击球，或者上半局还是下半局，很可能同时为右边投球手投球，而右边投球手投球的同时击球的击球手在哪个方向和此时是上半局或是下半局没有较强的关联性

因此说明，本次对atbats.csv数据集挖掘出的关联规则价值较低，即整个数据集各个属性之间几乎完全独立，没有强相关关系。